# Bayesian Analysis

Coco Yu

### Set Up Environment

Set up parameters as variables

In [ ]:
(study <- params$study)

[1] "ema"

[1] "1day"

[1] 0

[1] "v5"

[1] "nested"

[1] "main"

Function conflicts

In [ ]:
#| message: false
#| warning: false
#| include: false
#| echo: false

# source
devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/fun_ml.R?raw=true")

ℹ SHA-1 hash of file is "77e91675366f10788c6bcb59fa1cfc9ee0c75281"

Chunk Defaults

In [ ]:
#| include: false

knitr::opts_chunk$set(attr.output='style="max-height: 500px;"')

options(tibble.width = Inf)
options(tibble.print_max = Inf)

Packages for script

In [ ]:
#| message: false
#| warning: false
#| include: false
#| echo: false

library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

── Attaching packages ────────────────────────────────────── tidymodels 1.1.0 ──
✔ broom        1.0.5          ✔ rsample      1.1.1     
✔ dials        1.2.0          ✔ tune         1.1.1     
✔ infer        1.0.4          ✔ workflows    1.1.3     
✔ modeldata    1.1.0          ✔ workflowsets 1.0.1     
✔ parsnip      1.1.0.9004     ✔ yardstick    1.2.0     
✔ recipes      1.0.6          
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.

Source support functions

In [ ]:
#| message: false
#| warning: false
#| include: false
#| echo: false

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")

ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

Absolute paths

In [ ]:
path_fairema <- format_path(str_c("studydata/risk/data_processed/fairema"))

### Bayesian Analysis

In [ ]:
contrast_table <- function(df, met){
  df |> 
    filter(group != "all") |>
    select(c(met, "group", "outer_split_num")) |> 
    group_by(group) |>
    mutate(repeat_num = trunc((outer_split_num - 1) / 10 + 1),
           fold_num = outer_split_num %% 10) |> 
    select(c(met, "group", "repeat_num", "fold_num")) |>
    pivot_wider(values_from = c(met), names_from = group) |>
    rename(id = repeat_num, id2 = fold_num) |>
    perf_mod(formula = statistic ~ model + (1 | id2/id),
             # prior_intercept = rstanarm::student_t(autoscale = TRUE),
             # prior = rstanarm::student_t(autoscale = TRUE),
             # transform = tidyposterior::logit_trans,  # for skewed & bounded AUC
             # iter = 2000, chains = 4,
             adapt_delta = .99,
             # cores = 4, seed = 12345,
             family = gaussian, verbose = FALSE) |> 
    contrast_models() |> 
    summary(size = 0) |> 
    mutate(metrics = met, .before = contrast)
  }

In [ ]:
df_gender <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "gender", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                      col_types = cols())

rbind(
  contrast_table(df_gender, "auroc"),
  contrast_table(df_gender, "sensitivity"),
  contrast_table(df_gender, "specificity"),
  contrast_table(df_gender, "ppv"),
  contrast_table(df_gender, "balaccuracy")
)

ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(met)

  # Now:
  data %>% select(all_of(met))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000694 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 6.94 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.933 seconds (Warm-up)
Chain 1:                0.862 seconds (Sampling)
Chain 1:                2.7

# A tibble: 5 × 10
  metrics     contrast       probability   mean    lower  upper  size pract_neg
  <chr>       <chr>                <dbl>  <dbl>    <dbl>  <dbl> <dbl>     <dbl>
1 auroc       Male vs Female       0.996 0.0484  0.0191  0.0777     0        NA
2 sensitivity Male vs Female       0.780 0.0354 -0.0443  0.115      0        NA
3 specificity Male vs Female       0.960 0.0513  0.00348 0.0979     0        NA
4 ppv         Male vs Female       0.691 0.0174 -0.0453  0.0811     0        NA
5 balaccuracy Male vs Female       0.980 0.0439  0.00921 0.0799     0        NA
  pract_equiv pract_pos
        <dbl>     <dbl>
1          NA        NA
2          NA        NA
3          NA        NA
4          NA        NA
5          NA        NA

In [ ]:
df_race <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "race", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                    col_types = cols())

rbind(
  contrast_table(df_race, "auroc"),
  contrast_table(df_race, "sensitivity"),
  contrast_table(df_race, "specificity"),
  contrast_table(df_race, "ppv"),
  contrast_table(df_race, "balaccuracy")
)


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 3.1e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.31 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.241 seconds (Warm-up)
Chain 1:                0.63 seconds (Sampling)
Chain 1:                1.871

# A tibble: 5 × 10
  metrics     contrast           probability   mean   lower upper  size
  <chr>       <chr>                    <dbl>  <dbl>   <dbl> <dbl> <dbl>
1 auroc       white vs non_white       1     0.113   0.0681 0.157     0
2 sensitivity white vs non_white       0.976 0.0790  0.0136 0.144     0
3 specificity white vs non_white       1     0.153   0.0791 0.224     0
4 ppv         white vs non_white       0.757 0.0326 -0.0452 0.112     0
5 balaccuracy white vs non_white       1     0.116   0.0786 0.154     0
  pract_neg pract_equiv pract_pos
      <dbl>       <dbl>     <dbl>
1        NA          NA        NA
2        NA          NA        NA
3        NA          NA        NA
4        NA          NA        NA
5        NA          NA        NA

In [ ]:
df_income <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "income", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                      col_types = cols())

rbind(
  contrast_table(df_income, "auroc"),
  contrast_table(df_income, "sensitivity"),
  contrast_table(df_income, "specificity"),
  contrast_table(df_income, "ppv"),
  contrast_table(df_income, "balaccuracy")
)


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 3.5e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.35 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.445 seconds (Warm-up)
Chain 1:                0.505 seconds (Sampling)
Chain 1:                1.95

# A tibble: 5 × 10
  metrics     contrast                       probability    mean    lower
  <chr>       <chr>                                <dbl>   <dbl>    <dbl>
1 auroc       below_poverty vs above_poverty     0.00375 -0.0774 -0.121  
2 sensitivity below_poverty vs above_poverty     0.001   -0.169  -0.249  
3 specificity below_poverty vs above_poverty     0.912    0.0394 -0.00945
4 ppv         below_poverty vs above_poverty     0.669    0.0214 -0.0592 
5 balaccuracy below_poverty vs above_poverty     0.004   -0.0643 -0.104  
    upper  size pract_neg pract_equiv pract_pos
    <dbl> <dbl>     <dbl>       <dbl>     <dbl>
1 -0.0333     0        NA          NA        NA
2 -0.0869     0        NA          NA        NA
3  0.0900     0        NA          NA        NA
4  0.105      0        NA          NA        NA
5 -0.0250     0        NA          NA        NA

In [ ]:
df_age <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "age", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                   col_types = cols())

rbind(
  contrast_table(df_age, "auroc"),
  contrast_table(df_age, "sensitivity"),
  contrast_table(df_age, "specificity"),
  contrast_table(df_age, "ppv"),
  contrast_table(df_age, "balaccuracy")
)


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 3.2e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.32 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 1.311 seconds (Warm-up)
Chain 1:                0.825 seconds (Sampling)
Chain 1:                2.13

# A tibble: 5 × 10
  metrics     contrast         probability    mean   lower    upper  size
  <chr>       <chr>                  <dbl>   <dbl>   <dbl>    <dbl> <dbl>
1 auroc       older vs younger      0.0005 -0.0940 -0.133  -0.0551      0
2 sensitivity older vs younger      0.0318 -0.0747 -0.140  -0.00763     0
3 specificity older vs younger      0.0145 -0.0704 -0.122  -0.0197      0
4 ppv         older vs younger      0.330  -0.0178 -0.0847  0.0509      0
5 balaccuracy older vs younger      0.0005 -0.0733 -0.109  -0.0381      0
  pract_neg pract_equiv pract_pos
      <dbl>       <dbl>     <dbl>
1        NA          NA        NA
2        NA          NA        NA
3        NA          NA        NA
4        NA          NA        NA
5        NA          NA        NA